## Ingestion of Imperviousness 2018 10m raster
https://sdi.eea.europa.eu/catalogue/srv/eng/catalog.search#/metadata/3bf542bd-eebd-4d73-b53c-a0243f2ed862

https://land.copernicus.eu/pan-european/high-resolution-layers/imperviousness/status-maps/imperviousness-density-2018

In [26]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline

import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import rasterio
from time import process_time
# Start the stopwatch / counter 
t1_start = process_time() 


from f01_ingestion_utils import *
# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")
    
    
list_byoc_collections()

# Stop the stopwatch / counter
t1_stop = process_time()
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", t1_stop-t1_start)


Collection name: urban_audit_2021_city
Collection id:  20774d38-7b2e-43e9-b574-66fa2954da1c
-------------------
Collection name: UrbanAtlas2018_10m_raster_demo
Collection id:  55c49191-4afa-4384-956e-e9b746f259f3
-------------------
Collection name: nuts3_2021
Collection id:  81a17892-38ff-470a-ae46-b64313acd7b3
-------------------
Collection name: UrbanAtlas2012_10m_raster_test2
Collection id:  974d338a-3814-4b83-b2f8-ee57f93c87c3
-------------------
Collection name: urban_audit_2021_fua
Collection id:  b17f90da-1308-439e-b675-6e3a87f883de
-------------------
Collection name: UrbanAtlas2012_10m_raster_test
Collection id:  b5a24968-e26a-42f4-bab6-09a9d38af697
-------------------
Collection name: nuts3_2021
Collection id:  ed7c0bf4-e85f-498c-ad18-f3e83fb89a6f
-------------------
Collection name: UrbanAtlas2012_10m_raster
Collection id:  f0585df9-c1af-4da7-b01d-c614bc129533
-------------------
Collection name: TreeCoverDensity2018_10m_raster
Collection id:  f268b6be-1224-4b8b-9534-466c3f

In [20]:
# Initialize SentinelHubBYOC class
#byoc = SentinelHubBYOC(config=config)
#col_delete = byoc.get_collection("f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08")
#byoc.delete_collection(col_delete)

''

In [27]:
# Initialize SentinelHubBYOC class
# Start the stopwatch / counter 
t1_start = process_time() 
byoc = SentinelHubBYOC(config=config)

new_collection = ByocCollection(name="ImperviousnessDensity2018_10m_raster", s3_bucket="hub-fairicube0")
#byoc.delete_collection(new_collection) # for the delte of the collection
created_collection = byoc.create_collection(new_collection)
my_collection = byoc.get_collection(created_collection["id"])
# my_collection

#https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html#Create-new-collection
#Name: Name: data/d009_Imperviousness/imd2018/
## use ALT-SHIFT-mouse for changing B01 into (BAND)!!!!
input_folder ="data/d009_Imperviousness/imd2018"

tile_list = [
        "IMD_2018_010m_eu_03035_V2_0_1_1_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_1_2_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_1_3_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_1_4_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_1_5_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_1_6_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_1_7_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_2_1_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_2_2_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_2_3_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_2_4_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_2_5_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_2_6_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_2_7_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_3_1_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_3_2_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_3_3_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_3_4_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_3_5_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_3_6_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_3_7_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_4_1_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_4_2_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_4_3_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_4_4_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_4_5_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_4_6_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_4_7_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_5_1_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_5_2_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_5_3_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_5_4_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_5_5_(BAND).tif",
        "IMD_2018_010m_eu_03035_V2_0_5_6_(BAND).tif"

            ]

# ingest tiles
for tile in tile_list:
    path_link = input_folder+"/"+tile
    new_tile = ByocTile(path=path_link) 
    created_tile = byoc.create_tile(created_collection, new_tile)
    tile = byoc.get_tile(collection=created_collection["id"], tile=created_tile["id"])
    print(tile["status"])
# path_link = input_folder
# new_tile = ByocTile(path=path_link) 
# created_tile = byoc.create_tile(created_collection, new_tile)
# tile = byoc.get_tile(collection=created_collection["id"], tile=created_tile["id"])
# Stop the stopwatch / counter
t1_stop = process_time()
print("Elapsed time:", t1_stop, t1_start) 
print("Elapsed time during the whole program in seconds:", t1_stop-t1_start)

WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
WAITING
Elapsed time: 12.783091619 10.384888699
Elapsed time during the whole program in seconds: 2.398202920000001


In [30]:
#############################################################
#name_of_your_collection = name_of_your_collection
name_of_your_collection = "ImperviousnessDensity2018_10m_raster"
#############################################################

print ("START....")
# Initialize SentinelHubBYOC class
byoc = SentinelHubBYOC(config=config)
print ("START--------------------------")
# list collections and tiles
# from: https://sentinelhub-py.readthedocs.io/en/latest/examples/byoc_request.html


collections_iterator = byoc.iter_collections(name_of_your_collection)
my_collections = list(collections_iterator)[0]


data_collection = DataCollection.define_byoc(my_collections["id"])
data_collection

#'UrbanAtlas2018_10m_raster_v1'
#Collection id:  27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
#DataCollection.BYOC_27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
print("Collection name:", my_collections["name"])
print("Collection id: ", my_collections["id"])
tiles = list(byoc.iter_tiles(my_collections))
for tile in tiles:
        print("Tile status: ", tile['status'])
        print("Tile created: ", tile['created'])
        

print ("END----------------------------------------------------------")



START....
START--------------------------
Collection name: ImperviousnessDensity2018_10m_raster
Collection id:  94c0149f-131a-4c0c-8809-11aeab5da253
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:48.635584Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:47.512466Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:48.226668Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:47.405373Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:50.472373Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:48.024590Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:49.444122Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:49.646128Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:49.141298Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:49.745553Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:50.367370Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:50.046685Z
Tile status:  INGESTED
Tile created:  2023-06-13T08:15:49.0

In [25]:
#byoc.delete_collection(my_collections)


#xx error hello world
##for collection in my_collections:   
 # byoc.delete_collection(collection)
#print ("end")

tiles = list(byoc.iter_tiles(collection))
for tile in tiles:
    print("Tile status: ", tile['status'])
    print("Tile created: ", tile['created'])
    if(tile['status'] == "FAILED"):
        print("Ingestion failed error: ", tile['additionalData'])
print("-------------------")

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/byoc/collections/f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08/tiles?
with HTTPError:
404 Client Error: Not Found for url: https://services.sentinel-hub.com/api/v1/byoc/collections/f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08/tiles
Server response: "{"status": 404, "reason": "Not Found", "message": "Collection f27dd2b0-dba3-407e-bbe9-39e2cf8d9c08 not found.", "code": "COMMON_NOT_FOUND"}"